# 03 • Process ZuCo (v1+v2)
Load unified ET+EEG token-level table and show QA.

In [ ]:
from pathlib import Path
from pcs_toolbox.zuco import load_all
df, qa = load_all(write_outputs=True)
print('Rows:', len(df))
df.head(10)

In [ ]:
import json
print(json.dumps(qa.get('summary', qa), indent=2))